<a href="https://colab.research.google.com/github/amcheyre-nw/DL_Object_detection/blob/main/YOLOmain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import json
import urllib
import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split

# Python and ML Libraries
import os
import random
import numpy as np
import pandas as pd
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2
import json

# xml library for parsing xml files
from xml.etree import ElementTree as et


import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

from xml.etree import ElementTree as et

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# these are the helper libraries imported.
#from engine import train_one_epoch, evaluate
import utils
#import transforms as T

# for image augmentations (not working this)
# import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2
np.random.seed(42)

In [ ]:
pip install utils

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install dependencies
%cd yolov5

import torch
from IPython.display import Image, clear_output

Cloning into 'yolov5'...
remote: Enumerating objects: 11142, done.
remote: Total 11142 (delta 0), reused 0 (delta 0), pack-reused 11142
Receiving objects: 100% (11142/11142), 11.14 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (7703/7703), done.
     |████████████████████████████████| 596 kB 7.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
/content/yolov5


In [ ]:
import os
import json
import argparse
from tqdm import tqdm



def bdd2coco_detection(id_dict, labeled_images, fn):

    images = list()
    annotations = list()

    counter = 0
    for i in tqdm(labeled_images):
        counter += 1
        image = dict()
        image['file_name'] = i['name']
        image['height'] = 720
        image['width'] = 1280

        image['id'] = counter

        empty_image = True

        for label in i['labels']:
            annotation = dict()
            category=label['category']
            if (category == "traffic light"):
                color = label['attributes']['trafficLightColor']
                category = "tl_" + color
            if category in id_dict.keys():
                empty_image = False
                annotation["iscrowd"] = 0
                annotation["image_id"] = image['id']
                x1 = label['box2d']['x1']
                y1 = label['box2d']['y1']
                x2 = label['box2d']['x2']
                y2 = label['box2d']['y2']
                annotation['bbox'] = [x1, y1, x2-x1, y2-y1]
                annotation['area'] = float((x2 - x1) * (y2 - y1))
                annotation['category_id'] = id_dict[category]
                annotation['ignore'] = 0
                annotation['id'] = label['id']
                annotation['segmentation'] = [[x1, y1, x1, y2, x2, y2, x2, y1]]
                annotations.append(annotation)

        if empty_image:
            continue

        images.append(image)

    attr_dict["images"] = images
    attr_dict["annotations"] = annotations
    attr_dict["type"] = "instances"

    print('saving...')
    json_string = json.dumps(attr_dict)
    with open(fn, "w") as file:
        file.write(json_string)


if __name__ == '__main__':
    
    label_dir='/content/drive/MyDrive/bdd100k/BDD100K'
    save_path='/content/drive/MyDrive/bdd100k/BDD100K'

    attr_dict = dict()
    attr_dict["categories"] = [
        {"supercategory": "none", "id": 1, "name": "person"},
        {"supercategory": "none", "id": 2, "name": "rider"},
        {"supercategory": "none", "id": 3, "name": "car"},
        {"supercategory": "none", "id": 4, "name": "bus"},
        {"supercategory": "none", "id": 5, "name": "truck"},
        {"supercategory": "none", "id": 6, "name": "bike"},
        {"supercategory": "none", "id": 7, "name": "motor"},
        {"supercategory": "none", "id": 8, "name": "tl_green"},
        {"supercategory": "none", "id": 9, "name": "tl_red"},
        {"supercategory": "none", "id": 10, "name": "tl_yellow"},
        {"supercategory": "none", "id": 11, "name": "tl_none"},
        {"supercategory": "none", "id": 12, "name": "traffic sign"},
        {"supercategory": "none", "id": 13, "name": "train"}
    ]

    attr_id_dict = {i['name']: i['id'] for i in attr_dict['categories']}

    # create BDD training set detections in COCO format
    print('Loading training set...')
    with open(os.path.join(label_dir,
                           'bdd100k_labels_images_train.json')) as f:
        train_labels = json.load(f)
    print('Converting training set...')

    out_fn = os.path.join(save_path,
                          'bdd100k_labels_images_det_coco_train.json')
    bdd2coco_detection(attr_id_dict, train_labels, out_fn)

    print('Loading validation set...')
    # create BDD validation set detections in COCO format
    with open(os.path.join(label_dir,
                           'bdd100k_labels_images_val.json')) as f:
        val_labels = json.load(f)
    print('Converting validation set...')

    out_fn = os.path.join(save_path,
                          'bdd100k_labels_images_det_coco_val.json')
    bdd2coco_detection(attr_id_dict, val_labels, out_fn)

Loading training set...


FileNotFoundError: ignored

In [ ]:
import os
from xml.etree.ElementTree import dump
import json
import pprint

import argparse

from Format import COCO,YOLO

def main(config):

    if config["datasets"] == "COCO":
        coco = COCO()
        yolo = YOLO(os.path.abspath(config["cls_list"]))

        flag, data = coco.parse(config["label"])

        if flag == True:
            flag, data = yolo.generate(data)

            if flag == True:
                flag, data = yolo.save(data, config["output_path"], config["img_path"],
                                        config["img_type"], config["manipast_path"])

                if flag == False:
                    print("Saving Result : {}, msg : {}".format(flag, data))

            else:
                print("YOLO Generating Result : {}, msg : {}".format(flag, data))

        else:
            print("COCO Parsing Result : {}, msg : {}".format(flag, data))

    else:
        print("Unkwon Datasets")

if __name__ == '__main__':

#     config ={
#         "datasets": "COCO",
#         "img_path": "bdd100k_images/bdd100k/images/100k/train",
#         "label": "labels/bdd100k_labels_images_det_coco_train.json",
#         "img_type": ".jpg",
#         "manipast_path": "./",
#         "output_path": "labels/trains/",
#         "cls_list": "data/bdd100k.names",
#     }
    config ={
        "datasets": "COCO",
        "img_path": "bdd100k_images/bdd100k/images/100k/val",
        "label": "/content/drive/MyDrive/bdd100k/BDD100K/bdd100k_labels_images_det_coco_val.json",
        "img_type": ".jpg",
        "manipast_path": "./",
        "output_path": '/content/drive/MyDrive/bdd100k/BDD100K',
        "cls_list": "/content/drive/MyDrive/bdd100k/BDD100K/bdd100k.names.txt",
    }

    main(config)

In [ ]:

path_val_img = '/content/drive/MyDrive/bdd100k/data/val'
img_list = os.listdir(path_val_img)

In [ ]:
path_val_labels = '/content/drive/MyDrive/bdd100k/data/labels'
label_list = os.listdir(path_val_labels)

In [ ]:
label_name = []
for label in label_list:
  label_name.append(label.split(".")[0])

In [ ]:
val_path = '/content/drive/MyDrive/bdd100k/data/val/'
import shutil
for img in img_list:
  img_name = img.split(".")[0]
  if img_name in label_name:
    file = val_path + img
    shutil.copy(file, '/content/drive/MyDrive/bdd100k/data/img')

In [ ]:
 import glob, os, shutil

'''
Sometimes your image data set might not match with your label data set.
This code does the folowing
(1) Go through your image data set
(2) Search if the corresponding label file exist in the label data set. 
(3) If not, remove current image
'''


def copy_filter(label_dir,image_dir,target_dir_images,target_dir_labels):
    for image in os.listdir(image_dir):
        if image.endswith('jpg'):
            image_name = os.path.splitext(image)[0]

            # Corresponding label file name
            label_name = image_name + '.txt'
            image_path = image_dir + '/' + image_name + '.jpg'
            #print('ip.  ',image_path)
            if os.path.isfile(label_dir  + label_name) == False:
                print(" -- DELETE IMAGE [Label file not found -- ]")
                
                print(image_path)
#                 os.remove(image_path)
#           else:
                target_images=target_dir_images+ '/' + image_name + '.jpg'
                shutil.copy(image_path,target_dir_images )
                print(" --COPY IMAGE "+target_images)


    for label in os.listdir(label_dir):
        if label.endswith('.txt'):
            label_name = os.path.splitext(label)[0]

            # Corresponding label file name
            image_name = label_name + '.jpg'
            label_path = label_dir + '/' + label_name + '.txt'
            if os.path.isfile(image_dir + image_name) == False:
                print(" -- DELETE LABEL [Image file not found -- ]")
                print(label_path)
#                 os.remove(label_path)
#           else:
                target_labels=target_dir_labels+ '/' + label_name + '.txt'
                shutil.copy(label_path,target_labels )
                print(" --COPY lABELS "+target_labels)

In [ ]:
label_dir = '/content/drive/MyDrive/bdd100k/data/labels'
image_dir = '/content/drive/MyDrive/bdd100k/data/img'
target_dir_images="/content/drive/MyDrive/bdd100k/data/target"
target_dir_labels="/content/drive/MyDrive/bdd100k/data/target"
copy_filter(label_dir,image_dir,target_dir_images,target_dir_labels)


 -- DELETE IMAGE [Label file not found -- ]
/content/drive/MyDrive/bdd100k/data/img/b2b32394-e46669cb.jpg
 --COPY IMAGE /content/drive/MyDrive/bdd100k/data/target/b2b32394-e46669cb.jpg
 -- DELETE IMAGE [Label file not found -- ]
/content/drive/MyDrive/bdd100k/data/img/b1f6c103-b00e8aad.jpg
 --COPY IMAGE /content/drive/MyDrive/bdd100k/data/target/b1f6c103-b00e8aad.jpg
 -- DELETE IMAGE [Label file not found -- ]
/content/drive/MyDrive/bdd100k/data/img/b1e1a7b8-0aec80e8.jpg
 --COPY IMAGE /content/drive/MyDrive/bdd100k/data/target/b1e1a7b8-0aec80e8.jpg
 -- DELETE IMAGE [Label file not found -- ]
/content/drive/MyDrive/bdd100k/data/img/b2bdb7b6-041bebc9.jpg
 --COPY IMAGE /content/drive/MyDrive/bdd100k/data/target/b2bdb7b6-041bebc9.jpg
 -- DELETE IMAGE [Label file not found -- ]
/content/drive/MyDrive/bdd100k/data/img/b1e62c91-eca210a9.jpg
 --COPY IMAGE /content/drive/MyDrive/bdd100k/data/target/b1e62c91-eca210a9.jpg
 -- DELETE IMAGE [Label file not found -- ]
/content/drive/MyDrive/bdd100k/

In [ ]:
!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex


Cloning into 'apex'...
remote: Enumerating objects: 9057, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 9057 (delta 69), reused 45 (delta 28), pack-reused 8929
Receiving objects: 100% (9057/9057), 14.57 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (6182/6182), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
User install by explicit request
Created temporary directory: /tmp/pip-ephem-wheel-cache-3rb9pk_1
Created temporary directory: /tmp/pip-req-tracker-gc6qk985
Initialized build tracking at /tmp/pip-req-tracker-gc6qk985
Created build tracker: /tmp/pip-req-tracker-gc6qk985
Entered build tracker: /tmp/pip-req-tracker-gc6qk985
Created temporary d

In [2]:
 !git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install dependencies
%cd yolov5

import torch
from IPython.display import Image, clear_output

Cloning into 'yolov5'...
remote: Enumerating objects: 11142, done.
remote: Total 11142 (delta 0), reused 0 (delta 0), pack-reused 11142
Receiving objects: 100% (11142/11142), 11.14 MiB | 34.46 MiB/s, done.
Resolving deltas: 100% (7703/7703), done.
     |████████████████████████████████| 596 kB 5.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
/content/yolov5


In [ ]:
pip install -U PyYAML

In [ ]:
#!python train.py --img 640 --batch 16 --epochs 5 --data ./data/coco128.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name tutorial --nosave --cache


train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=./data/coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=16, imgsz=640, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=tutorial, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degree

In [3]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/bdd100k/data/bddk.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name tutorial --nosave --cache


train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/drive/MyDrive/bdd100k/data/bddk.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=tutorial, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv